# PHYS 6260: Homework 5, C. Michael Haynes

In [ ]:
# generic list of import statements to not have to keep track
import numpy as np
from scipy import constants
import math as m
import warnings
import os
import sys
from random import random,randrange

# importing specific to animation formalism found in the template notebook
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
plt.rcParams['animation.html'] = 'html5' # this is used to display animations in jupyter notebooks

from mpl_toolkits.axes_grid1 import make_axes_locatable

warnings.filterwarnings( "ignore")

## Problem 1
### Monte Carlo Integration
We wish to use the Importance Sampling technique to evaluate 
$$ I = \int_{a=0}^{b=1} \frac{x^{-\frac{1}{2}}}{e^x + 1} \, \mathrm{d}x \qquad .$$
This method uses a non-uniform random sample to evaluate the integral without encountering evaluation issues near the point where the function diverges (i.e., the origin). 

#### (a) Choice of Probability Distribution $p(x)$

As discussed in lecture, this is acheived by selecting a _weighting function_ $w(x)$ that factors out the diverging product. Since the function 
$$ f(x) = \frac{x^{-\frac{1}{2}}}{e^x + 1}$$
diverges on $[a,b]$ due to the $x^{-\frac{1}{2}}$ factor, we choose $w(x) \equiv x^{-\frac{1}{2}}$ such that 
$$ \frac{f(x)}{w(x)} = \frac{ \frac{x^{-\frac{1}{2}}}{e^x + 1}}{x^{-\frac{1}{2}}} = \frac{1}{e^x + 1} \qquad ,$$
which is well-behaved.

Now, since this choice of the weighting function $w$ eliminates the divergent product, we can determine a probability distribution based on the formalism outlined in the lecture slides (specifically, slide set 09: pg 14):
$$ p(x) = \frac{w(x)}{\int_a^bw(x)\mathrm{d}x} = \frac{x^{-\frac{1}{2}}}{\int_0^1 x^{-\frac{1}{2}}\mathrm{d}x} \qquad .$$
When the integral in the denominator is evaluated, this simply becomes
$$ p(x) = \frac{1}{2x^{\frac{1}{2}}} \qquad ,$$ 
as found in the prompt. 

We map the image of $p(x)$ to the interval $[0,1]$ via
$$ M(p(x)) \mapsto \frac{p(x)}{1+p(x)} $$
so that
$$\DeclareMathOperator{\Ima}{Im} \Ima{M} \sim \frac{\Ima{p}}{\Ima{p}+1} \mapsto [0,1] \qquad .$$
Thus, by selecting numbers $x_i$ from $p(x)$ and passing them through the map $M$, we can obtain a weighted set of $x$ values of arbitrary size on the interval $[0,1]$. 

Another choice is the function
$$ M_2(x) = \frac{2\arctan{x}}{\pi} $$ 
with accompanying inverse mapping
$$ M_2^{-1} = \tan{\left ( \frac{\pi x}{2} \right ) }  \qquad .$$

#### (b) Evaluate the Integral
To do so, we will implement a numerical routine to approximate the integral of $f(x)$ over $[a,b]$ with the expression obtained in lecture using the Importance Sampling formalism:
$$ I \approx \frac{1}{N} \sum_{i=1}^N \frac{f(x_i)}{w(x_i)} \int_a^b w(x) \,\mathrm{d}x \qquad , $$
for a MC process with $N$ samples. Substituting our expressions for $f(x),\,w(x),$ we have:
$$ I \approx \frac{1}{N} \sum_{i=1}^N \frac{2}{e^{x_i} + 1} \qquad , $$ 
where the sample points $x_i$ are obtained from $M(p(x))$ 

In [ ]:
# define probability function p(x)
def p(x):
    return (1./(2.*np.sqrt(x)))

# define mappings M, M2 that go between the domains [0,1] and [0, infty)
def M(x):
    return (x/(x+1.))

def Minv(x):
    return (x/(1.-x))

def M2(x):
    return (2. * np.arctan(x) / np.pi)

def Minv2(x):
    return (np.tan(np.pi * x / 2.))

# define function to calculate the integral of the Fermi distribution
# inputs: N samples, mapping M and inverse mapping Minv
# since Minv maps [0,1] to [0,infty), we use this to initialize our xi values

def IS_Fermi(N,M,Minv):
    xi_ar = []
    for j in range(N):
        xi_ar.append(np.random.random())
    xi_ar = np.array(xi_ar)
    xi_arr = Minv(xi_ar)
    pxi_arr = p(xi_arr)
    Mpxi_arr = M(pxi_arr)
    eval_arr = 2. / (1.+np.exp(Mpxi_arr))
    Ninv = 1./N
    return (Ninv * eval_arr.sum())


def pk(k,N,M,Minv):
    vals = []
    for i in range(k):
        vals.append(IS_Fermi(N,M,Minv))
    stats = np.array(vals)
    for i in range(k):
        print('sample '+str(i+1)+': '+str(IS_Fermi(N,M,Minv)))
    print('mean: '+str(np.mean(stats)))

print('--------------\nFive N=1e6 samples using polynomial mapping M:\n--------------')
pk(5,int(1e6),M,Minv)
print('--------------\nFive N=1e6 samples using trigonometric mapping M2:\n--------------')
pk(5,int(1e6),M2,Minv2)

## Problem 2
### Randomizing a vector bearing
As shown in the homework prompt, the probability of a randomly selected point on a sphere falling within any particular element is given by:

$$p(\theta,\phi)\,\mathrm{d}\theta\,\mathrm{d}\phi = \frac{\sin{\theta\,\mathrm{d}\theta\,\mathrm{d}\phi}}{4\pi} \qquad ,$$ 

which can be decomposed into a product of single variable functions:

$$p(\theta,\phi)\,\mathrm{d}\theta\,\mathrm{d}\phi = \frac{\sin{\theta \,\mathrm{d}\theta}}{2} \cdot \frac{\mathrm{d} \phi}{2\pi} = p(\theta)\,\mathrm{d}\theta \cdot p(\phi)\,\mathrm{d}\phi \qquad .$$


#### (a) Distributions for $\theta,\phi$

The variables $\theta$ and $\phi$ represent colatitude and longitude, respectively. Longitude covers $360^\circ$ and latitude only $180^\circ$, so the ranges of the two variables are given by:
$$\theta\in[0,\pi] \qquad , \qquad \phi\in[0,2\pi] \qquad .$$

The normalization can thus be verified directly via integration. For $\theta$:
$$ \int_0^\pi \frac{sin{\theta}}{2} \,\mathrm{d}\theta = \frac{1}{2} \left ( -\cos{\theta} \big\vert_0^\pi \right ) = - \frac{1}{2} (-1 -1) = 1 $$
For $\phi$, it is trivial:
$$\int_0^{2\pi} \frac{1}{2\pi} \,\mathrm{d}\phi = \frac{2\pi}{2\pi} = 1 \qquad . $$



#### (b) Randomly Sampling the distributions $p(\theta),\,p(\phi)$

As stated, the $\phi$ one is trivial since this is a uniform distribution over the interval $[0,2\pi]$. Thus, a randomly selected $\phi$ would merely be a random number on the unit interval multiplied by $2\pi$. 

$$ P(\phi) = \frac{\phi}{2\pi} $$

For $\theta$, however, the case is less clear. Since this is _not_ a uniform distribution, we need to obtain the inverse CDF to transform a uniformly generated random number into a value from $p(\theta)$. This can be obtained via integration:

$$ P(\theta) = \int_0^\theta p(\theta') \,\mathrm{d}\theta' = \int_0^\theta  \frac{\sin{\theta'}}{2}  \,\mathrm{d}\theta' $$
$$ P(\theta) = \frac{1-\cos{\theta}}{2} $$

Then, by inverting these CDFs we can obtain an expression mapping a random number on the unit interval to these PDFs determined by our spherical geometry. Doing so yields:

$$ \phi = 2 \pi r $$
$$ \theta = \arccos{(1-2r)} $$

#### (c) Routine for random point generator

In [ ]:
# function for inverse CDF of phi
def f_phi(r):
    return (2. * np.pi * r)

# function for inverse CDF of theta
def f_theta(r):
    return (m.acos(1. - 2. * r))

# function for generating a random point on a sphere
def rand_point_sphere(radius):
    r_mag = radius
    r1, r2 = np.random.random(),np.random.random()
    phi = f_phi(r1)
    theta = f_theta(r2)
    x = r_mag * m.sin(theta) * m.cos(phi)
    y = r_mag * m.sin(theta) * m.sin(phi)
    z = r_mag * m.cos(theta)
    return np.array([x,y,z])

x = rand_point_sphere(1.)


# brielfy plot for sanity, as shown in starter code
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(x[0], x[1], x[2], c='r', marker='o')
# plt.show()
    
    

#### (d) Routine for random point mesh

In [ ]:
# helper function to stabalize the 3D plot viewing


def set_axes_equal(ax):
    x_limits = ax.get_xlim3d()
    y_limits = ax.get_ylim3d()
    z_limits = ax.get_zlim3d()
    x_range = abs(x_limits[1] - x_limits[0])
    x_mid = np.mean(x_limits)
    y_range = abs(y_limits[1] - y_limits[0])
    y_mid = np.mean(y_limits)
    z_range = abs(z_limits[1] - z_limits[0])
    z_mid = np.mean(z_limits)
    
    plot_radius = (1./2.)*max([x_range, y_range, z_range])
    
    ax.set_xlim3d([x_mid - plot_radius, x_mid + plot_radius])
    ax.set_ylim3d([y_mid - plot_radius, y_mid + plot_radius])
    ax.set_zlim3d([z_mid - plot_radius, z_mid + plot_radius])


# define arrays to store entries for each coordinate
X, Y, Z = [],[],[]
# iterate 500 instances
for i in range(500):
    k_vec = rand_point_sphere(1.)
    X.append(k_vec[0])
    Y.append(k_vec[1])
    Z.append(k_vec[2])

# plot, as shown in the starter code
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X, Y, Z, c='r', marker='o')

ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
set_axes_equal(ax)
plt.title('Random points on a globe of radius 1')

plt.show()

## NOTE: I was unable to activate the ipympl library on my machine

## Problem 3
### Application Question


Many systems studied in continuum physics exhibit processes that can be modeled with Monte Carlo (MC) methods. One example of such processes is the behavior of a plasma, for instance, as it impinges towards an obstacle like the Earth's (or another planet's) magnetosphere. For objects with or without a strong internal magnetic field, this flow couples strongly to the ionosphere of the object: it generates production of ions, modifies / shapes the current systems, and likewise contributes to its outflow and loss. 

In some plasma simulation codes, the ion motion occurs on scales large enough to produce asymmetries in this interaction, so many numerical approaches include a _kinetic_ treatment of ions with a fluid (MHD) treatment of electrons: this is known as a hybrid model. Ionospheric production occurs through chemical reactions. These individual particle interactions, such as charge exchange or photo-ionization via solar uv rays, are calculated in many hybrid models using a MC process like the one we used in the Rutherford scattering example from lecture. The probability of the interaction can be computed with the interaction cross section. Thus, many aspects of ionospheric generation, current balance, and loss can be attributed to a net result of many small-scale MC simulations. 

Moreover, some numerical applications can substantiate the use of a MC approach, too. For example, in many hybrid and/or kinetic plasma models, particles are traced as _macroparticles_: that is, they represent an ensemble of particles near a certain coordinate in phase space. These are traced together and sampled from distribution functions obtained through measurement, when possible. These macroparticles thus represent different (usually large) numbers of real particles. The number of macroparticles to trace, then, is a choice to be made by the simulation. It is possible to specify an "optimum" number of macroparticles in each grid node, and then use merging / splitting processes (which approximately conserve COM, energy and momentum) to adjust the number of macroparticles in each cell after the particles are advanced and some may cross over into adjacent grid cells. The coordinates with which to initialize these particles needs to be sampled from a distribution using a MC approach. 